In [ ]:
def default(tel):
  return f"""
lvm.{tel}.pwi:
    ra_j2000_hours: {tel}:ra_j2000_h
    dec_j2000_degs: {tel}:dec_j2000_d
    altitude_degs: {tel}:altitude_d
    azimuth_degs: {tel}:azimuth_d

lvm.{tel}.foc:
    Position: {tel}:foc_dt

lvm.{tel}.km:
    Position: {tel}:km_d
    SkyPA: {tel}:sky_d

lvm.tel:
    temperature: bentemp_inside
    humidity_enclosure: benhumid_outside
    temperature_enclosure: bentemp_outside
    dewpoint_enclosure: bendewpoi_outside

lvm.{tel}.agcam:
    east.temperature: {tel}:east.temp
    east.filename: {tel}:east.file
    west.temperature: {tel}:west.temp
    west.filename: {tel}:west.file
    center.temperature: {tel}:center.temp
    center.filename: {tel}:center.file
"""


tel = ['sci', 'skye', 'skyw', 'spec']


In [ ]:
import yaml
import asyncio
from lvmtipo.scraper import Scraper

import numpy as np
import pandas as pd

from functools import reduce
from cluplus.proxy import flatten

from ipydatagrid import TextRenderer, DataGrid


config = {t: yaml.safe_load(default(t)) for t in tel}
key2index = {k.split(":")[-1]: i for i, k in enumerate(flatten(config['sci']).values())}

df = pd.DataFrame(
    data={
        "Key": key2index.keys(),
        **{t: [''] * len(key2index.keys()) for t in tel}
    }
)
default_renderer = TextRenderer(
    text_elide_direction="left",
)

grid = DataGrid(df, index_name="", default_renderer=default_renderer)
grid.auto_fit_columns = True
grid.auto_fit_params = {"area": "body", "padding": 120, "numCols": 5}
display(grid)

client = await Scraper(reduce(lambda a, b: dict(a, **b), config.values())).start()
log = client.log

while(True):
        for k, v in client.scraper_store.items():
            sk = k.split(':')
            grid.set_cell_value(sk[0] if len(sk) > 1 else 'sci', key2index[sk[-1]] ,v[0])
        await asyncio.sleep(1.0)
